# Amazon SageMaker MLOps: from idea to production in six steps

This sequence of six notebooks takes you from developing your ML idea in a simple notebook to a production solution with automated model building and CI/CD deployment pipelines, and model monitoring.

Follow these steps one by one:
1. Experiment in a notebook
2. Move to SageMaker SDK for data processing and training
3. Add an ML pipeline, a model registry, a feature store
4. Add a model building CI/CD pipeline
5. Add a model deployment CI/CD pipeline
6. Add data quality monitoring

![](img/six-steps.png)

There are also additional hands-on examples of other SageMaker features and ML topics, like [A/B testing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html), custom [processing](https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html), [training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) and [inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) containers, [debugging and profiling](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html), [security](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html), [multi-model](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) and [multi-container](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html) endpoints, and [serial inference pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). Explore the notebooks in the folder `additional-topics` to test out these features.

To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

## Star GitHub repository

In [1]:
%%html

<a class="github-button" href="https://github.com/aws-samples/amazon-sagemaker-from-idea-to-production" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Amazon SageMaker secure MLOps on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

### Click this button ^^^ above ^^^

## Setup
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 The workshop and all notebooks were tested on the SageMaker Python SDK (the package sagemaker) version 2.219.0. The notebooks don't pin the version of the sagemaker. If you encounter any incompatibility issues, you can install the specific version of the sagemaker by running the pip command: <code>%pip install sagemaker=2.219.0</code>
</div>

In [2]:
# Uncomment if you have any compatibility issues and would like to use the specific version of the sagemaker library
# %pip install sagemaker==2.219.0
%pip install --upgrade pip sagemaker boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 82.6 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.131
    Uninstalling botocore-1.34.131:
      Successfully uninstalled botocore-1.34.131
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.131
    Uninstalling boto3-1.34.131:
      Successfully uninstalled boto3-1.34.131
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.224.1
    Uninstalling sagemaker-2.224.1:
      Successfully uninstalled sagemaker-2.224.1
ERROR: pip's dependency resolver does

In [3]:
%pip install mlflow==2.13.2 sagemaker-mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 75.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Import packages

In [2]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


('2.226.1', '1.34.146')

### Set constants

In [3]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "from-idea-to-prod/xgboost"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()
dataset_file_local_path = "data/bank-additional/bank-additional-full.csv"

initialized = True

print(sm_role)

arn:aws:iam::398333718380:role/cfnstudiodomain-SageMakerExecutionRole-8XQVuLUFsn6g


In [5]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized
%store dataset_file_local_path

Stored 'bucket_name' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)
Stored 'dataset_file_local_path' (str)


### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value. The following code demonstrates how to access the notebook metadata file and get the `domain_id`.

In [6]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

%store domain_id
%store space_name
%store user_profile_name

SageMaker domain id: d-wxmppmljqkj5
Space name: sagemaker-space
User profile: studio-user-e44b6ca0
Stored 'domain_id' (str)
Stored 'space_name' (str)
Stored 'user_profile_name' (str)


### Connect to MLflow tracking server
If you're running an AWS-led workshop or used the delivered CloudFormation template to provision your workshop environment, an MLflow server must be up and running. If you don't have an MLflow server, follow the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) or run the following code cell to create a new one.

To create and manage an MLflow tracking server and to work with managed MLflow experiements, you need the following permissions attached to the SageMaker execution role:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

Execute the following code to check if you have a running MLflow server.

In [7]:
# Find an active MLflow server in the account
r = boto3.client("sagemaker").list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

if len(r) < 1:
    print("You don't have any running MLflow servers. Trying to find a server in the status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_arn = r[0]['TrackingServerArn']
    mlflow_name = r[0]['TrackingServerName']
    print(f"You have {len(r)} running MLflow server(s). Get the first server ARN:{mlflow_arn}")

You have 1 running MLflow server(s). Get the first server ARN:arn:aws:sagemaker:us-east-1:398333718380:mlflow-tracking-server/mlflow-d-wxmppmljqkj5


In [8]:
# This code cell creates a new MLflow server
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{domain_id}-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Server creation request succeded. The server {mlflow_arn} is being created.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
Creation of an MLflow server can take up to 25 minutes. You don't need to wait - proceed with the flow of the workshop.
</div>

In [9]:
(mlflow_arn, mlflow_name)

('arn:aws:sagemaker:us-east-1:398333718380:mlflow-tracking-server/mlflow-d-wxmppmljqkj5',
 'mlflow-d-wxmppmljqkj5')

In [10]:
%store mlflow_arn
%store mlflow_name

Stored 'mlflow_arn' (str)
Stored 'mlflow_name' (str)


## Install Docker to enable Studio local mode
Amazon SageMaker Studio applications support the use of local mode to create estimators, processors, and pipelines, then deploy them to a local environment. With local mode, you can test machine learning scripts before running them in Amazon SageMaker managed training or hosting environments. Refer to [Local mode support in Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html) to understand which docker operations the Studio currently supports.

To use local mode in Studio applications, you must install Docker into your JupyterLab space. 

### Check if docker access is enabled

In [35]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id)['DomainSettings'].get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

The docker access is ENABLED in the domain d-wxmppmljqkj5


<div style="border: 4px solid coral; text-align: center; margin: auto;">
If the previous code cell raised an exeption that the docker access is not enabled, you need to enable the access. See the following instructions how to do it.
</div>

In [31]:
print(f"Domain id: {domain_id}")

Domain id: d-wxmppmljqkj5


### Enable docker access for the SageMaker domain

<div class="alert alert-info">You only need this section if the docker access is not enabled in the domain.
</div>

You need `sagemaker:UpdateDomain` permission in the execution role. You cannot update domain from this notebook because the notebook execution role doesn't have this permission. To update domain settings, you can use one of the following options.

#### Option 1: run `update_domain` in the notebook
If you have the corresponding permissions in the notebook execution role, you can run the following code in a notebook:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
        }
    }
)
```

#### Option 2: run `aws sagemaker` CLI in the  terminal
Make sure you run `AWS CLI` in the terminal where you have the corresponding permissions `sagemaker:UpdateDomain`. Run the following command:

```
aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update DockerSettings={EnableDockerAccess='ENABLED'}
```

For example, you can run the command above in the [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/) in your AWS account.

In [36]:
# check the updated settings
sm_client.describe_domain(DomainId=domain_id)['DomainSettings']

{'DockerSettings': {'EnableDockerAccess': 'ENABLED',
  'VpcOnlyTrustedAccounts': []}}

### Install Docker

In [12]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2717 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2070 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2791 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1418 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2335 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [33.7 kB]
Fetched 12.9 MB in 1s (10.9 MB/s)
Reading package lists...
Reading package lists...
Building dependency tree..

debconf: delaying package configuration, since apt-utils is not installed


Fetched 1341 kB in 1s (1859 kB/s)
Selecting previously unselected package openssl.
(Reading database ... 13791 files and directories currently installed.)
Preparing to unpack .../openssl_3.0.2-0ubuntu1.16_amd64.deb ...
Unpacking openssl (3.0.2-0ubuntu1.16) ...
Selecting previously unselected package ca-certificates.
Preparing to unpack .../ca-certificates_20230311ubuntu0.22.04.1_all.deb ...
Unpacking ca-certificates (20230311ubuntu0.22.04.1) ...
Setting up openssl (3.0.2-0ubuntu1.16) ...
Setting up ca-certificates (20230311ubuntu0.22.04.1) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
Updating certificates in /etc/ssl/certs...
rehash: warning: skipping ca-certificates.crt,it does not contain exactly one certificate or CRL
137 added, 0 removed; done.
Processing triggers for ca-certific

debconf: delaying package configuration, since apt-utils is not installed


Fetched 55.7 MB in 1s (88.6 MB/s)
Selecting previously unselected package docker-ce-cli.
(Reading database ... 14258 files and directories currently installed.)
Preparing to unpack .../docker-ce-cli_5%3a20.10.24~3-0~ubuntu-jammy_amd64.deb ...
Unpacking docker-ce-cli (5:20.10.24~3-0~ubuntu-jammy) ...
Selecting previously unselected package docker-compose-plugin.
Preparing to unpack .../docker-compose-plugin_2.29.0-1~ubuntu.22.04~jammy_amd64.deb ...
Unpacking docker-compose-plugin (2.29.0-1~ubuntu.22.04~jammy) ...
Setting up docker-compose-plugin (2.29.0-1~ubuntu.22.04~jammy) ...
Setting up docker-ce-cli (5:20.10.24~3-0~ubuntu-jammy) ...
Client: Docker Engine - Community
 Version:           20.10.24
 API version:       1.41
 Go version:        go1.19.7
 Git commit:        297e128
 Built:             Tue Apr  4 18:21:03 2023
 OS/Arch:           linux/amd64
 Context:           default
 Experimental:      true

Server:
 Engine:
  Version:          20.10.25
  API version:      1.41 (minimum 

## Data

This example uses the [direct marketing dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing) from UCI's ML Repository:
> [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

Download and unzip the dataset:

In [13]:
!wget -P data/ -N https://archive.ics.uci.edu/static/public/222/bank+marketing.zip --no-check-certificate

--2024-07-23 07:48:55--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/bank+marketing.zip’

bank+marketing.zip      [            <=>     ] 999.85K  64.0KB/s    in 15s     

Last-modified header missing -- time-stamps turned off.
2024-07-23 07:49:10 (67.3 KB/s) - ‘data/bank+marketing.zip’ saved [1023843]



In [14]:
import zipfile

with zipfile.ZipFile("data/bank+marketing.zip", "r") as z:
    print("Unzipping bank+marketing...")
    z.extractall("data")

with zipfile.ZipFile("data/bank-additional.zip", "r") as z:
    print("Unzipping bank-additional...")
    z.extractall("data")

print("Done")

Unzipping bank+marketing...
Unzipping bank-additional...
Done


### See the data

In [15]:
df_data = pd.read_csv(dataset_file_local_path, sep=";")

pd.set_option("display.max_columns", 500)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


### Upload data to S3

In [16]:
input_s3_url = sagemaker.Session().upload_data(
    path=dataset_file_local_path,
    bucket=bucket_name,
    key_prefix=f"{bucket_prefix}/input"
)
print(f"Upload the dataset to {input_s3_url}")

%store input_s3_url

Upload the dataset to s3://sagemaker-us-east-1-398333718380/from-idea-to-prod/xgboost/input/bank-additional-full.csv
Stored 'input_s3_url' (str)


## Restart kernel

In [17]:
# Restart kernel to get the packages
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Further workshop flow
You can continue the workshop by going through each of the following notebooks in the direct order, e.g. 1-2-3-4....

If you're interested in a particular topic, you can execute some notebooks standalone, as shown in the following flow diagram:

![](img/workshop-flow.png)

Start with the step 1 [Idea Development](01-idea-development.ipynb) or step 3 [SageMaker Pipelines](03-sagemaker-pipeline.ipynb).

## Additional resources

### Documentation
- [Use Amazon SageMaker Built-in Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html)

### Hands-on examples
- [Get started with Amazon SageMaker](https://aws.amazon.com/sagemaker/getting-started/)


### Workshops
- [Amazon SageMaker 101 Workshop](https://catalog.us-east-1.prod.workshops.aws/workshops/0c6b8a23-b837-4e0f-b2e2-4a3ffd7d645b/en-US)